<a href="https://colab.research.google.com/github/KaziJahidurRahaman/lst-gee-py/blob/main/lst_gee_py_kutupalong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p> We are using <b>Geemap</b> for rendering the maps. Geemap is not preinstalled in the Google colab library. Therefor, we need to install geemap. If geemap is already installed. Please skip the next block. </p>

In [ ]:
!pip install geemap

In [3]:
# Import the required packages.

import os
import sys
import ee
import geemap

The to use the earthengine api, we need to authenticate using our google developer profile. We know more about the authentication process one can look at the following article. <a href = 'https://developers.google.com/earth-engine/guides/python_install'>Authentication in ee</a> 

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=reTd-_kyaCkhuIZiuEnECVTetwXEcWkyHMVAeejubkg&tc=8F2svX7eVUH4m7HBdx7Vj8oRVSEWbX4LbpWRzhlItbg&cc=0swCR6QiCUWQMrQ7Jn0-wj5OUy04MJtF4szMD4nV6Bk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvuYyl27FLXJuKgrDQx7TOM9GuGm8YNl8ABKjeIrf710oGjuRHAzyBo

Successfully saved authorization token.


In [5]:
# Initialize the ee library.
ee.Initialize()

Our area of interest data is loaded in google drive. so we will connect with the my google drive folder.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The are of interest is the <a href ='https://en.wikipedia.org/wiki/Kutupalong_refugee_camp'> Kutupalong Rohingya Camp </a>, Cox's Bazar, in Bangladesh. Where a huge deforestation has taken place over last 10 years for influx of Rohingya refugees. 

In [9]:
Map = geemap.Map()
aoi_shp =  '/content/drive/MyDrive/lst_files/kutupalong/Kutupalong-AOI-Polygon-polygon.shp'
aoi = geemap.shp_to_ee(aoi_shp)
Map.addLayer(aoi, {}, 'AOI')
Map.centerObject(aoi, zoom=16)
Map

Map(center=[21.198233064130157, 92.15051686742379], controls=(WidgetControl(options=['position', 'transparent_…

For the analysis we are using <a href = 'https://www.usgs.gov/landsat-missions/landsat-8#:~:text=Landsat%208%20images%20have%2015,km%20(115%20mi)%20swath.'>Landsat 8</a> data set. There are other satellite images from different satellites. However, the spatial resolution of Landsat 8 is more suitable for our analysis. For narowing down the search we will filter it using <a href = 'https://www.usna.edu/Users/oceano/pguth/md_help/html/landsat_path_row.html'>WRS ROW and PATH</a> and a time frame. 

In [30]:
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
image_collection = image_collection.filterMetadata('WRS_PATH','equals',136).filterMetadata('WRS_ROW', 'equals',45).filterDate('2021-01-01', '2021-12-31').sort('CLOUD_COVER')
print("Total Image Downloaded: ",image_collection.size().getInfo()) #gets the total number of images


Total Image Downloaded:  16


In [31]:
#for the anaylsis we are using  only the best image which has least cloud cover
least_cloudy_image = image_collection.first()

Before starting the analysis lets check the basic information for the image

In [32]:
least_cloudy_image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]},
  {'id': 'B6',
   'data_type': {'type': 'Pi

Visualize the sample image with geemap

In [33]:
Map.addLayer(least_cloudy_image)
Map.centerObject(least_cloudy_image)
Map

Map(bottom=58059.0, center=[20.88319225609666, 91.31586861360927], controls=(WidgetControl(options=['position'…

In [34]:
# Define the visualization parameters.

image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}
Map.addLayer(least_cloudy_image, image_viz_params, "Least Cloudy Image")
Map.centerObject(least_cloudy_image)
Map

Map(bottom=29026.0, center=[21.66600537458964, 91.61664634220797], controls=(WidgetControl(options=['position'…

In [35]:
#Picking the required bands
nir_band = least_cloudy_image.select('B5');
red_band = least_cloudy_image.select('B4');
thermal_band = least_cloudy_image.select('B10');

In [36]:
#cliping the band specific images for the AOI
nir_band_clipped = nir_band.clip(aoi)
red_band_clipped = red_band.clip(aoi)
thermal_band_clipped = thermal_band.clip(aoi);

Map.addLayer(nir_band_clipped,{},'clipped_NIR')
Map.centerObject(aoi)
Map.addLayerControl()
Map

Map(bottom=7422.0, center=[22.88756221517449, 79.85190840778414], controls=(WidgetControl(options=['position',…

In [37]:
# Compute the NDVI for clipped images/aoi
ndvi = (nir_band_clipped.subtract(red_band_clipped)).divide(  (nir_band_clipped.add(red_band_clipped)) ).rename('NDVI')

# visualize the ndvi
ndviPar = {
    'min': 0.4, 
    'max': 0.54, 
    'palette': ['red', 'yellow', 'green']
           }

Map.addLayer(ndvi, ndviPar, 'NDVI of AOI')
Map.centerObject(ndvi, zoom = 11)
Map


Map(bottom=3689100.0, center=[21.667712612943202, 91.61241607266717], controls=(WidgetControl(options=['positi…

In [38]:
ndvi.getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 241485, 0, -30, 2513715]}]}

In [39]:
##MIN NDVI VALUE

min_ndvi = ee.Number(ndvi.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print('min value of NDVI',min_ndvi.getInfo())

min value of NDVI -0.37243403175430867


In [40]:
max_ndvi = ee.Number(ndvi.reduceRegion(**{
"reducer": ee.Reducer.max(),
"scale": 30,
"maxPixels": 10**9
}).values().get(0));
print('max value of NDVI',max_ndvi.getInfo())

max value of NDVI 0.738098396337262


In [41]:
#proportional vegetation

pv =(ndvi.subtract(min_ndvi).divide(max_ndvi.subtract(min_ndvi))).pow(ee.Number(2)).rename('PV')

In [42]:
pv_max = ee.Number(pv.reduceRegion(**{
  "reducer": ee.Reducer.max(),
  "scale": 30,
  "maxPixels": 10**9
}).values().get(0));
print( 'max value of pv',pv_max.getInfo())

max value of pv 1


In [43]:
##MIN pv VALUE

min_pv = ee.Number(pv.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print('min value of pv',min_pv.getInfo())




min value of pv 0


In [46]:
Map.addLayer(pv, ndviPar, 'Proportional Vegetation')
Map.centerObject(pv, zoom = 11)
Map.center_object(aoi)
Map.add_layer(pv)
Map

Map(bottom=115572.0, center=[21.667712612943202, 91.61241607266717], controls=(WidgetControl(options=['positio…

In [49]:
#Emissivity
a= ee.Number(0.004);
b= ee.Number(0.986);
em=pv.multiply(a).add(b).rename('EM')

In [50]:
emmax = ee.Number(em.reduceRegion(**{
  "reducer": ee.Reducer.max(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(emmax.getInfo())

0.99


In [51]:
emmin = ee.Number(em.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(emmin.getInfo())

0.986


In [52]:
# Land surface temperature (LST) in C
LST = thermal_band_clipped.expression(
'(Tb/(1 + (10.8* (Tb / 14388))*log(EM)))-273.15', {
 'Tb': thermal_band_clipped,
'EM': em.select('EM')
}).rename('LST')


In [53]:
LSTParams = {
    'min':27.115292564360516, 
    'max':31.653865373204212, 
    'palette': ['gray','yellow','green']
    }

In [54]:
lstmin = ee.Number(LST.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(lstmin.getInfo())

24.61047603870543


In [55]:
lstmax = ee.Number(LST.reduceRegion(**{
  "reducer": ee.Reducer.max(),
  "scale": 30,
  "maxPixels": 10**9
  }).values().get(0))
print(lstmax.getInfo())

33.958986269391005


In [59]:
Map.addLayer(LST, LSTParams, 'LandSurfaceTemperature')
Map.center_object(aoi,zoom=10)
Map

Map(bottom=29118.0, center=[21.198233064130157, 92.15051686742379], controls=(WidgetControl(options=['position…

In [60]:
# // Export the image, specifying the CRS, transform, and region.
task = ee.batch.Export.image.toDrive(image=LST,
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     description='Kutupalong',
                                     folder='lst_files',
                                     maxPixels=1e9)

task.start()